In [126]:
import json #json 직렬화
import re # 정규표현식
import subprocess #외부 명령 실행에 사용?
from dis import disco
from re import match



In [79]:
from pathlib import Path #독립 경로 window,linux 호환좋음
from typing import List,Set,Optional,Iterable # 예 list 타입 고정할때

In [80]:
#fallback 사용한다 가정

try:
    import requests
except Exception:
    requests = None

In [131]:
#파일 생성
#raw 원본 그대로 저장
#   /data/raw 랑 data/raw 차이점 : /시작 하지않으면  현재 디렉토리 기준 , /있으면  cwd 최상위 디렉토리
ROW_DIR = Path("data/raw") #현재디랙토리
ROW_DIR.mkdir(parents=True,exist_ok=True)
#parents = 해당 경로에 폴더가 없으면 폴더생성
#exist_ok=True  해당 파일이 없으면 그냥 오류안내고 pass

In [82]:
#subfinder툴 : 보안 해킹(정보수집) 오픈소스 도메인/서브도메인

#해당 함수 목적 툴이 리눅스 안에  있는지?  //툴은 패키지가 아니다
#해당 함수는 리눅스 쉘,터미널 기준
def _which_(tool:str)-> bool: #tool타입은 왜 str 인가? 찾을때 이름 검색하는 생각
    try: #쉘-유틸 약자
        from shutil import which
        return which(tool) is not None
            #which 자체는 경로,none을 반환 하는데  (비교연산자 is not None)을 감싸서 True,False로 바뀜
    except Exception:
        return False
    

In [83]:
# cmd명령어 list[]로 받아 바이트->str 변환 입력
    #-os 터미널에서 str을 바이트로 인코딩하여 실행이됨.
    #파이썬에서 바이트 그냥 줘도 되지만 인코딩에서 error가 날수 있어서 str로 변환해서 보냄
#발견한 서브도메인 목록을 줄 단위로 출력(stdout)  <-- cmd에서 출력된 모든 도메인 보여줌

def _run(cmd : List[str],timeout=60) -> list[str]:
    try:
        out = subprocess.check_output(cmd,text=True,stderr=subprocess.DEVNULL,timeout=timeout)
                                        #text=True str로 유지               
        return [line.strip() for line in out.splitlines() if line.strip()]
                                                            # if line.strip() 해당 라인을 양쪽 공백을 없애서 남으면 True,안남으면 False
    except Exception:
        return []


In [84]:
#and보다 or이 먼저 계산됨
#도메인 주소 타겟의 서브 도메인 주소 매칭
def sub_domain_match(name:str,root:str) ->Optional[str]:
    
    n=name.lower() #소문자 변환
    n1=n.lstrip('*.') #예상되는 서브도메인 좌측에 *와일드 . 제거
    n2=n1.rstrip('.') # 예상되는 서브도메인 우측에 . 제거
    
    if not n2:
        return None
    
    if not n2.endswith('.'+root):
        if n2!=root:
            return None
        
    
    #불량문자 확인
    if re.search(r"[^a-z0-9-_.]",n2):
        n2 =re.sub(r"[^a-z0-9-_.]","",n2)
        
    return n2 or None
    

In [85]:
def de_dup(s:Iterable[str])->list[str]: #Iterable 범용 반복문 객체 set,list,파일
                                        #for line in file 예  전부 사용가능

    #s1= {line for line in s if line}   #중복제거  {}원래 key:value dic타입이나, value 없이 값만 오면 set타입이 된다.
    s1 = set(s)
    s2 = sorted(s1)   #sort 정렬 -ed 된
    
    return s2

In [92]:
def save_raw(domain:str,tool:str,raw:Iterable[str]) -> Path:#raw 가공되지 않은 범용데이터 계속 최신으로 교체됨
       
    path = ROW_DIR/f"{domain}_{tool}.json"
    
    data = json.dumps(json.dumps(raw),indent=2,ensure_ascii=False)
    path.write_text(data,encoding="utf-8") #ident 들여쓰기 ,ensure 보장해라
    #write_text 파일 생성및 덮어쓰기
    return path


In [93]:
def from_subfinder(domain:str)->list[str]:
    
    if not _which_("subfinder"):
        return None

    return _run(["subfinder","-d",domain,"-silent"])  #-d domain약자  , -silent 조용히

In [94]:
def from_amass_passive(domain:str)->list[str]:
    
    if not _which_("amass"):
        return None
    
    return _run(["amass","enum","-passive","-d",domain,"-silent"])
                        #enum 서브도메인 검색
                        #모드 -passive,-activte

#amass enum -passive -d domain -silent
#서버까지 도메인 주소 불러옴

In [120]:
def from_crt(domain:str,timeout: int=15)->list[str]:
    
    if requests is None:
        return None
    try:
        r = requests.get("https://crt.sh/",params={"q":domain,"output":"json"},timeout=timeout)
        r.raise_for_status() #응답이 실패상태인지 검사 get이 응답을 받았더라도 404,200 정상으로 반환하기 떄문
                
        #r이 request에서 객체가 안만들어 질때 예외
        #r 객체가 만들어 졌지만 서버에서 404 객체가 만들어 질때 r.raise_for_staus()로 상태확인 해야함
        
        #raise_for_status는 None 또는 httperror 리턴
        
        rows = r.json()
        names: list[str]=[]
        
        for row in rows:
            cn = (row.get("common_name") or "").strip()
            nv = (row.get("name_value")or "").strip()

            
            if cn:
                names.append(cn)
            
            if nv:
                names.extend([a for a in nv.split("\n") if a])
            
        return names
    
    except Exception:
        return [] #이미 리턴값을 리스트 형태로 약속 했기 떄문

In [123]:
#오케스트레이션

def discover(domain:str)->list[str]:
    
    
    raw:list[str]=[]
    
    sf =  from_subfinder(domain)
    if sf:
        
        save_raw(domain,"subfinder",sf)
        raw.extend(sf)
        
    am= from_amass_passive(domain)
    if am:
        
        save_raw(domain,"amass",am)
        raw.extend(am)
        
    
        
    if not raw:
        
        ct =from_crt(domain)
        if ct:
            save_raw(domain,"crt",ct)
            raw.extend(ct)
    
    
    raw1_match = [sub_domain_match(name,domain) for name in raw if sub_domain_match(name,domain)]
    raw_result = de_dup(raw1_match) 
    
    
    save_raw(domain,"final_domain",raw_result)
    return  raw_result
    
    
   

In [125]:
if __name__=='__main__':
    target = "tryhackme.com"
    
    subs = discover(target)
    print(f"[subdomains] {target}: {len(subs)} found")  #window 쥬피터 환경이라 웹 api crt밖에 작동안함
    for s in subs[:]:
        print(" -", s)

[subdomains] tryhackme.com: 23 found
 - assets.tryhackme.com
 - auth.tryhackme.com
 - blog.tryhackme.com
 - business.tryhackme.com
 - cancel.tryhackme.com
 - careers.tryhackme.com
 - docs.tryhackme.com
 - guacaworker.tryhackme.com
 - help.tryhackme.com
 - insights-proxy-worker.tryhackme.com
 - lp.tryhackme.com
 - monitoring.tryhackme.com
 - newhelp.tryhackme.com
 - remote-eu-1.tryhackme.com
 - remote-us-west-1.tryhackme.com
 - remote.tryhackme.com
 - resources.tryhackme.com
 - reverse-proxy-eu-west-1.tryhackme.com
 - reverse-proxy-us-east-1.tryhackme.com
 - setup.auth.tryhackme.com
 - store.tryhackme.com
 - tryhackme.com
 - www.tryhackme.com


In [133]:
import re
try:
    import requests
except Exception:
    requests = None
import json
from pathlib import Path
from typing import Iterable,
import subprocess


In [196]:
import re
try:
    import requests
except Exception:
    requests = None
import json
from pathlib import Path
from typing import Iterable,Optional
import subprocess

class Sub_domain_search:
    def __init__(self,domain:str):
        self.ROW_DIR = Path("data/raw")
        self.ROW_DIR.mkdir(parents=True,exist_ok=True)
        self.domain = domain
        
    def _which_(self,tool:str)->bool:
        
        try:
            from shutil import which
            return which(tool) is not None
        
        except Exception:
            return False
        
    def _run_(self,cmd:list[str],timeout=60)->list[str]:
        try:
            out =subprocess.check_output(cmd,text=True,stderr=subprocess.DEVNULL,timeout=timeout)
            return [line for line in out.splitlines() if line.strip()]
        
        except Exception:
            return []
            
    def _domain_normalize_(self,subdomain:str)->Optional[str]:
        
        sub1=subdomain.lower().strip()
        sub2=sub1.lstrip('*.')
        sub2=sub2.rstrip('.')
        
        if not sub2:
            return None
        if  not (sub2==self.domain or sub2.endswith('.'+self.domain)):
            
            return None
        
        if re.search(r"[^a-z0-9-_.]",sub2): #True 이것 외에 문자가 있는 경우
            sub2=re.sub(r"[^a-z0-9-_.]","",sub2)
        
        
        return  sub2 or None
    
    def _ddep_(self,data:Iterable[str])->list[str]:
        data1= set(data)
        data2=sorted(data1)
        return data2
    
    def _save_raw_(self,tool:str,raw:Iterable[str])->Path:
        try:
            path = self.ROW_DIR/f"{self.domain}_{tool}.json"
            data = json.dumps(list(raw),indent=2,ensure_ascii=False)
            path.write_text(data,encoding="utf-8")
        
        except Exception:
            return None

        return path
    
    def from_subfinder(self)->list[str]:
        
        if not self._which_("subfinder"):
            return []
        
        raw =  self._run_(["subfinder","-d",self.domain,"-silent"])
        return raw
    
    def from_amass_passive(self)->list[str] :
        
        if not self._which_("amass"):
           return []
        
        raw = self._run_(["amass","enum","-passive","-d",self.domain,"-silent"])
        return raw
    
    def from_crt(self)->list[str]:
        
        try:
            r=requests.get("https://crt.sh/",params={"q":self.domain,"output":"json"},timeout=60)
            r.raise_for_status()
            
            data = r.json()
            names: list[str]=[]
            
            for line in data:
                cn =(line.get("common_name") or "").strip()
                nv = (line.get("name_value") or "").strip()
                
                if cn:
                    names.append(cn)
            
                if nv:
                    names.extend([line for line in nv.split("\n") if line])
            
            return names
        
        except Exception:
            return []
        
        
        
     
    def discover(self)->list[str]:
         
        raw:list[str]=[]
        sf = self.from_subfinder()
         
        if sf:
            self._save_raw_("subfinder",sf)
            raw.extend(sf)
        
        am = self.from_amass_passive()
        if am:
            self._save_raw_("amass",am)
            raw.extend(am)
            
        if not raw:
            
            ct =self.from_crt()
            if ct:
                self._save_raw_("crt",ct)
                raw.extend(ct)
                
    
    
        match= (self._domain_normalize_(ra) for ra in raw)
        result = self._ddep_(ma for ma in match if ma)
        self._save_raw_("final_domain",result)
        
        return result
    
    def __call__(self):
    
        return self.discover()
        
        
         

In [197]:
target = "tryhackme.com"

s=Sub_domain_search(target)
subs= s()                # __call__ → discover() 실행
print(f"[subdomains] {target}: {len(subs)} found")
for su in subs[:20]:
    print(" -", su)

[subdomains] tryhackme.com: 23 found
 - assets.tryhackme.com
 - auth.tryhackme.com
 - blog.tryhackme.com
 - business.tryhackme.com
 - cancel.tryhackme.com
 - careers.tryhackme.com
 - docs.tryhackme.com
 - guacaworker.tryhackme.com
 - help.tryhackme.com
 - insights-proxy-worker.tryhackme.com
 - lp.tryhackme.com
 - monitoring.tryhackme.com
 - newhelp.tryhackme.com
 - remote-eu-1.tryhackme.com
 - remote-us-west-1.tryhackme.com
 - remote.tryhackme.com
 - resources.tryhackme.com
 - reverse-proxy-eu-west-1.tryhackme.com
 - reverse-proxy-us-east-1.tryhackme.com
 - setup.auth.tryhackme.com
